# Regresión lineal Múltiple

## Utiliza el método de optimización de Eliminación hacia Atrás PASO A PASO (Manualmente)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### Importar el dataset

In [2]:
dataset = pd.read_csv("50_Startups.csv")

In [3]:
X = dataset.iloc[:, :-1].values

In [4]:
y = dataset.iloc[:, 4].values

### Codificar datos categóricos

In [5]:
labelencoder_X = LabelEncoder()

In [6]:
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])

In [7]:
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [3])],   
    remainder='passthrough'                        
)

In [8]:
X = np.array(ct.fit_transform(X), dtype=np.float)

### Evitar la trampa de las variables ficticias (Dummy)

In [9]:
X = X[:, 1:]

### Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Ajustar el modelo de Regresión Lineal Múltiple con el conjunto de datos de entrenamiento

In [11]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [12]:
print(f'Los coeficientes de las variables independientes son {regression.coef_}')

Los coeficientes de las variables independientes son [-9.59284160e+02  6.99369053e+02  7.73467193e-01  3.28845975e-02
  3.66100259e-02]


In [13]:
print(f'El intercepto de la recta con el eje Y es {regression.intercept_}')

El intercepto de la recta con el eje Y es 42554.16761772438


### El modelo de regresión múltiple creado es:

$$\tilde{y} = -9.59284160e^{2}\cdot x_{1} + 6.99369053e^{2}\cdot x_{2} + 7.73467193e^{-1}\cdot x_{3} + 3.28845975e^{-2}\cdot x_{4} + 3.66100259e^{-2}\cdot x_{5} + 42554.16761772438 $$

### Cálculo del error cuadrático medio (MSE) de los *y_train* respecto a los *y* del modelo de predicción

$$MSE = \frac{\sum_{i=1}^n(y_i - \tilde{y_i})^2}{n}$$

In [14]:
mean_squared_error(y_train, regression.predict(X_train))

81571001.8007737

### Cálculo del coeficiente de determinación (r^2) de los *y_train* respecto a los *y* del modelo de predicción

In [15]:
r2_score(y_train, regression.predict(X_train))

0.9501847627493607

## Testing

### Predecir los datos (Regresión Lineal Múltiple) con el conjunto de datos de test

In [16]:
y_pred = regression.predict(X_test)

### Cálculo del error cuadrático medio (MSE) de los *y_test* respecto a los *y* del modelo de predicción

In [17]:
mean_squared_error(y_test, y_pred)

83502864.03257468

### Cálculo del coeficiente de determinación (r^2) de los *y_test* respecto a los *y* del modelo de predicción

In [18]:
r2_score(y_test, y_pred)

0.9347068473282446

# --------------------------------------------------------------------------------------------------------------

# Construir el modelo óptimo de Regresión Lineal Múltiple utilizando la Eliminación hacia Atrás

<img src="Eliminacion hacia atras.png">

In [28]:
import statsmodels.api as sm

### Añadir nueva columna de unos (1) para incluir el termino independiente b0, se añade al inicio de X

In [29]:
X = np.append(arr = np.ones((50, 1)).astype(int), values = X, axis = 1)

### 1er paso: Creamos el nivel de significación SL

In [21]:
SL = 0.05

### 2do paso: Se inicia la optimización con todas la variables independientes

In [22]:
X_opt = X[:, [0, 1, 2, 3, 4, 5]].tolist()

### Crear el OLS (Ordinary List Square - Mínimos cuadrados ordinarios) con todas las variables predictoras

In [23]:
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 14 Jul 2021   Prob (F-statistic):           1.34e-27
Time:                        23:36:27   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x2           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3er y 4o Paso: Eliminamos la variable independiente x2 con el p-valor mas grande ya que es mayor que SL y ajustamos nuevamente

In [24]:
X_opt = X[:, [0, 1, 3, 4, 5]].tolist()
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Wed, 14 Jul 2021   Prob (F-statistic):           8.49e-29
Time:                        23:39:58   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
x1           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3er y 4o Paso: Eliminamos la variable independiente x1 con el p-valor mas grande ya que es mayor que SL y ajustamos nuevamente

In [30]:
X_opt = X[:, [0, 3, 4, 5]].tolist()
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     278.7
Date:                Wed, 14 Jul 2021   Prob (F-statistic):           1.68e-29
Time:                        23:50:09   Log-Likelihood:                -526.81
No. Observations:                  50   AIC:                             1062.
Df Residuals:                      46   BIC:                             1069.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.507e+04   6145.947      8.960      0.000    4.27e+04    6.74e+04
x1          -573.7029   2838.043     -0.202      0.841   -6286.386    5138.981
x2             0.8624      0.030     28.282      0.000       0.801       0.924
x3            -0.0530      0.050     -1.063      0.294      -0.154       0.047
==============================================================================
Omnibus:                       14.902   Durbin-Watson:                   1.199
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.212
Skew:                          -0.964   Prob(JB):                     2.48e-05
Kurtosis:                       5.543   Cond. No.                     6.74e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.74e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3er y 4o Paso: Eliminamos la variable independiente x4 con el p-valor mas grande ya que es mayor que SL y ajustamos nuevamente

In [26]:
X_opt = X[:, [0, 3, 5]].tolist()
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 14 Jul 2021   Prob (F-statistic):           2.16e-31
Time:                        23:41:19   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 3er y 4o Paso: Eliminamos la variable independiente x5 con el p-valor mas grande ya que es mayor que SL y ajustamos nuevamente

In [27]:
X_opt = X[:, [0, 3]].tolist()
regression_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regression_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Wed, 14 Jul 2021   Prob (F-statistic):           3.50e-32
Time:                        23:41:55   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# CONCLUSIÓN: Para este caso, la variable R&D es la variable más estadísticamente significativa